In [1]:
import os
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.cityscapes import CityscapesCustom
from models.deeplabv2.deeplabv2 import get_deeplab_v2
import torch.optim as optim

In [2]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
#Create Dataloaders for Cityscapes
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

train_dataset = CityscapesCustom(cityscapes_dir, 'train')
test_dataset = CityscapesCustom(cityscapes_dir, 'val')

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f'Train: {len(train_dataset)} images, divided into {len(train_dataloader)} batches of size {train_dataloader.batch_size}')
print(f'Test: {len(test_dataset)} images, divided into {len(test_dataloader)} batches of size {test_dataloader.batch_size}')

Train: 1572 images, divided into 1572 batches of size 1
Test: 500 images, divided into 500 batches of size 1


In [4]:
#Set up the model with the pretrained weights
model = get_deeplab_v2().to(device)

Deeplab pretraining loading...


In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [6]:
#Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [7]:
#Define the training loop
def train(model, criterion, optimizer, train_loader, test_loader, epochs, device):    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for image, label in train_loader:
            image, label = image.to(device), label.to(device)
            #print(f'Image type: {image.dtype}, Image shape: {image.shape}')
            #print(f'Label type: {label.dtype}, Label shape: {label.shape}')

            optimizer.zero_grad()
            output, _, _ = model(image)
            #print(f'Output type: {output.dtype}, Output shape: {output.shape}')

            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for image, label in test_loader:
                image, label = image.to(device), label.to(device)
                output = model(image)
                loss = criterion(output, label)
                test_loss += loss.item()
            test_loss /= len(test_loader)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

In [8]:
#Train the model
train(model, criterion, optimizer, train_dataloader, test_dataloader, 1, device)

Image type: torch.float32, Image shape: torch.Size([1, 3, 1024, 2048])
Label type: torch.float32, Label shape: torch.Size([1, 1, 1024, 2048])


KeyboardInterrupt: 